# ARMA Forecasting on Monthly Power Consumption

This notebook demonstrates ARMA(2,2) modeling, forecasting, and diagnostics on differenced monthly power consumption data.

## 1. Import Libraries

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.metrics import mean_absolute_error, mean_squared_error

warnings.filterwarnings("ignore")
plt.rcParams["figure.figsize"] = (12,6)


## 2. Load Dataset

In [ ]:

# Replace with your dataset path
df = pd.read_csv("power_consumption.csv", parse_dates=["Date"], index_col="Date")

# Differencing to ensure stationarity
df["Power_Consumption_diff"] = df["Power_Consumption"].diff().dropna()
df = df.dropna()
df.head()


## 3. Train-Test Split

In [ ]:

train_size = int(len(df) * 0.8)
train = df.iloc[:train_size]["Power_Consumption_diff"]
test = df.iloc[train_size:]["Power_Consumption_diff"]

print("Train size:", len(train))
print("Test size:", len(test))


## 4. Fit ARMA(2,2) Model

In [ ]:

model = ARIMA(train, order=(2,0,2))
arma_result = model.fit()
print(arma_result.summary())


## 5. Forecasting

In [ ]:

forecast = arma_result.forecast(steps=len(test))


## 6. Forecast Visualization

In [ ]:

plt.plot(train.index, train, label="Train")
plt.plot(test.index, test, label="Actual", linestyle="--")
plt.plot(test.index, forecast, label="Forecast", linestyle="--")
plt.title("ARMA Forecast Plot")
plt.legend()
plt.show()


## 7. Evaluation Metrics

In [ ]:

mae = mean_absolute_error(test, forecast)
rmse = np.sqrt(mean_squared_error(test, forecast))
mape = np.mean(np.abs((test - forecast) / test)) * 100

print("MAE :", mae)
print("RMSE:", rmse)
print("MAPE:", mape)


## 8. Residual Diagnostics

In [ ]:

residuals = arma_result.resid

plt.plot(residuals)
plt.title("Residuals")
plt.show()

lb_test = acorr_ljungbox(residuals, lags=[10], return_df=True)
lb_test


## Conclusion

The ARMA(2,2) model captures the linear dependence structure of the differenced power consumption series and produces reasonable short-term forecasts. Residual diagnostics confirm model adequacy, though extensions like SARIMA or ARIMAX may further improve performance.